In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
refunds = ScrapedRefund.objects.all().aggregate(Sum('amount'), Count('donor'))
print '%s in %s refunds' % (
    locale.currency(refunds['amount__sum'], grouping=True),
    locale.format('%d', refunds['donor__count'], grouping=True))

$1,080,898.01 in 2,074 refunds


In [3]:
groups = ScrapedContribution.objects.all().values('filing_committee__committee_group').annotate(sum=Sum('amount'), num=Count('donor'))
refund_groups = ScrapedRefund.objects.all().values('filing_committee__committee_group').annotate(sum=Sum('amount'), num=Count('donor'))
data = {}
for group in groups:
    data[group['filing_committee__committee_group']] = {
        'total_unadjusted': group['sum'],
        'num_contribs': group['num'],
        'total_adjusted': group['sum'],
        'num_refunds': 0
    }
for group in refund_groups:
    data[group['filing_committee__committee_group']]['total_adjusted'] -= group['sum']
    data[group['filing_committee__committee_group']]['num_refunds'] = group['num']
for idx, group in enumerate(sorted(data.keys(), key=lambda k: data[k]['total_adjusted'], reverse=True)):
    print '%d) %s: %s in %s contribs, after adjusting for %d refunds' % (
        idx + 1,
        group,
        locale.currency(data[group]['total_adjusted'], grouping=True),
        locale.format('%d', data[group]['num_contribs'], grouping=True),
        data[group]['num_refunds'])

1) continuing: $31,938,225.89 in 148,531 contribs, after adjusting for 231 refunds
2) federal: $27,866,759.76 in 99,186 contribs, after adjusting for 752 refunds
3) leg: $23,957,730.60 in 65,629 contribs, after adjusting for 373 refunds
4) sw: $19,690,264.81 in 68,811 contribs, after adjusting for 332 refunds
5) initiative: $18,234,812.99 in 32,767 contribs, after adjusting for 14 refunds
6) caucus: $13,831,086.38 in 2,077 contribs, after adjusting for 5 refunds
7) single_year: $10,514,167.22 in 16,932 contribs, after adjusting for 11 refunds
8) party_state: $8,627,977.85 in 5,646 contribs, after adjusting for 17 refunds
9) loc: $2,994,193.44 in 13,557 contribs, after adjusting for 26 refunds
10) party_county: $1,997,699.35 in 12,565 contribs, after adjusting for 31 refunds
11) jud: $1,981,879.32 in 9,055 contribs, after adjusting for 277 refunds
12) party_legdist: $667,318.96 in 5,907 contribs, after adjusting for 4 refunds
13) party_associated: $102,138.41 in 672 contribs, after adju

In [4]:
groups = ScrapedTotals.objects.exclude(
    filing_committee__office__icontains='president', filing_committee__committee_group='federal').values(
    'filing_committee__committee_group').annotate(sum=Sum('raised'))
data = {}
for group in groups:
    data[group['filing_committee__committee_group']] = {
        'total_raised': group['sum']
    }
for idx, group in enumerate(sorted(data.keys(), key=lambda k: data[k]['total_raised'], reverse=True)):
    print '%d) %s: %s raised (unitemized)' % (
        idx + 1,
        group,
        locale.currency(data[group]['total_raised'], grouping=True))

1) continuing: $46,337,811.25 raised (unitemized)
2) federal: $29,841,929.35 raised (unitemized)
3) leg: $28,528,453.05 raised (unitemized)
4) sw: $22,361,199.00 raised (unitemized)
5) initiative: $21,137,264.85 raised (unitemized)
6) caucus: $16,823,383.44 raised (unitemized)
7) single_year: $11,888,823.93 raised (unitemized)
8) party_state: $9,802,469.02 raised (unitemized)
9) loc: $3,694,013.57 raised (unitemized)
10) jud: $3,632,231.48 raised (unitemized)
11) party_county: $3,033,707.25 raised (unitemized)
12) party_legdist: $1,068,302.61 raised (unitemized)
13) party_associated: $152,264.37 raised (unitemized)
14) party_minor: $12,861.42 raised (unitemized)


In [5]:
groups = ScrapedContribution.objects.all().values('filing_committee__name').annotate(sum=Sum('amount'), num=Count('donor'))
refund_groups = ScrapedRefund.objects.all().values('filing_committee__name').annotate(sum=Sum('amount'), num=Count('donor'))
data = {}
for group in groups:
    data[group['filing_committee__name']] = {
        'total_unadjusted': group['sum'],
        'num_contribs': group['num'],
        'total_adjusted': group['sum'],
        'num_refunds': 0
    }
for group in refund_groups:
    try:
        #data[group['filing_committee__name']]['total_adjusted'] -= group['sum']
        data[group['filing_committee__name']]['num_refunds'] = group['num']
    except:
        print 'No group %s' % group['filing_committee__name']
print '\n'
for idx, group in enumerate(sorted(data.keys(), key=lambda k: data[k]['total_adjusted'], reverse=True)[:100]):
    print '%d) %s: %s in %s contribs, after adjusting for %d refunds' % (
        idx + 1,
        group,
        locale.currency(data[group]['total_adjusted'], grouping=True),
        locale.format('%d', data[group]['num_contribs'], grouping=True),
        data[group]['num_refunds'])

No group SAVE ANIMALS FACING EXTINCTION


1) INSLEE JAY R: $8,845,736.37 in 33,544 contribs, after adjusting for 27 refunds
2) MURRAY, PATTY: $8,170,930.74 in 15,614 contribs, after adjusting for 58 refunds
3) CLINTON, HILLARY RODHAM / KAINE, TIMOT: $7,069,181.29 in 63,200 contribs, after adjusting for 616 refunds
4) INTEGRITY WA: $4,015,457.41 in 1,617 contribs, after adjusting for 0 refunds
5) NEW DIRECTION PAC: $3,933,529.29 in 40 contribs, after adjusting for 0 refunds
6) ALLIANCE FOR GUN RESPONSIBILITY ERP COMM: $3,869,871.34 in 21,129 contribs, after adjusting for 9 refunds
7) MASS TRANSIT NOW: $3,668,361.00 in 345 contribs, after adjusting for 0 refunds
8) RAISE UP WA: $3,456,026.07 in 1,387 contribs, after adjusting for 1 refunds
9) BRYANT WILLIAM L: $3,442,661.12 in 8,824 contribs, after adjusting for 0 refunds
10) WA ST DEMO CENT COMM NON EXEMPT: $3,064,950.11 in 4,716 contribs, after adjusting for 17 refunds
11) WA ST REPUB PARTY NON EXEMPT: $2,960,194.00 in 581 contribs, af

In [6]:
def convert_party_code(party_code):
    mapping = {
        '': '(Blank)',
        'D': 'Democratic',
        'Democratic Party': 'Democratic',
        'R': 'Republican',
        'Republican Party': 'Republican',
        'O': 'Other',
        'Other': 'Other',
        'G': 'Green',
        'L': 'Libertarian',
        'I': 'Independent',
        'Independent': 'Independent',
        'N': 'Nonpartisan',
        'NO': 'Nonpartisan',
        'S': 'Socialist',
        'C': 'Constitutional',
        'Constitutional': 'Constitutional',
        'Green Party': 'Green',
        'None': 'Nonpartisan',
        'George Wallace Party': 'George Wallace Party',
        'Unknown': '(Blank)',
        'SOCIALIST EQUALITY PARTY': 'Socialist',
        'UNAFFILIATED': 'Nonpartisan',
        'WRITE-IN': 'Other',
        'Libertarian': 'Libertarian',
        'American Party': 'American',
        'NO PARTY AFFILIATION': 'Nonpartisan',
        'Peace Freedom Party': 'Peace Freedom',
        'CONSTITUTION PARTY': 'Constitution',
        'INDEPENDENCE PARTY': 'Independence'
    }
    if party_code not in mapping:
        print party_code
        return party_code
    return mapping[party_code]

groups = ScrapedContribution.objects.all().values('filing_committee__party').annotate(sum=Sum('amount'), num=Count('donor'))
refund_groups = ScrapedRefund.objects.all().values('filing_committee__party').annotate(sum=Sum('amount'), num=Count('donor'))
data = {}
for group in groups:
    if convert_party_code(group['filing_committee__party']) not in data:
        data[convert_party_code(group['filing_committee__party'])] = {
            'total_unadjusted': 0,
            'num_contribs': 0,
            'total_adjusted': 0,
            'num_refunds': 0
        }
    data[convert_party_code(group['filing_committee__party'])]['total_unadjusted'] += group['sum']
    data[convert_party_code(group['filing_committee__party'])]['total_adjusted'] += group['sum']
    data[convert_party_code(group['filing_committee__party'])]['num_contribs'] += group['num']
    
for group in refund_groups:
    try:
        data[convert_party_code(group['filing_committee__party'])]['total_adjusted'] -= group['sum']
        data[convert_party_code(group['filing_committee__party'])]['num_refunds'] = group['num']
    except:
        print 'No group %s' % group['filing_committee__party']
print '\n'
for idx, group in enumerate(sorted(data.keys(), key=lambda k: data[k]['total_adjusted'], reverse=True)[:20]):
    print '%d) %s: %s in %s contribs, after adjusting for %d refunds' % (
        idx + 1,
        group,
        locale.currency(data[group]['total_adjusted'], grouping=True),
        locale.format('%d', data[group]['num_contribs'], grouping=True),
        data[group]['num_refunds'])



1) Democratic: $62,893,018.15 in 199,363 contribs, after adjusting for 642 refunds
2) (Blank): $62,669,085.42 in 207,285 contribs, after adjusting for 533 refunds
3) Republican: $34,628,628.58 in 64,439 contribs, after adjusting for 23 refunds
4) Other: $1,312,631.14 in 4,407 contribs, after adjusting for 8 refunds
5) Nonpartisan: $860,969.33 in 5,608 contribs, after adjusting for 1 refunds
6) Libertarian: $35,822.36 in 225 contribs, after adjusting for 1 refunds
7) Green: $8,552.75 in 118 contribs, after adjusting for 1 refunds
8) Independent: $4,100.00 in 8 contribs, after adjusting for 0 refunds


In [7]:
groups = ScrapedCommittee.objects.filter(party='').values('committee_group').annotate(num=Count('name'))
print groups

[{'num': 30, 'committee_group': u'initiative'}, {'num': 195, 'committee_group': u'jud'}, {'num': 547, 'committee_group': u'continuing'}, {'num': 162, 'committee_group': u'single_year'}]


In [8]:
groups = ScrapedTotals.objects.exclude(
    filing_committee__office__icontains='president', filing_committee__committee_group='federal').values(
    'filing_committee__party').annotate(
    sum_raised=Sum('raised'), sum_spent=Sum('spent'), sum_ie_for=Sum('ie_for'), sum_ie_against=Sum('ie_against'))
data = {}
for group in groups:
    if convert_party_code(group['filing_committee__party']) not in data:
        data[convert_party_code(group['filing_committee__party'])] = {
            'total_raised': 0,
            'total_spent': 0,
            'total_ie_for': 0,
            'total_ie_against': 0
        }
    data[convert_party_code(group['filing_committee__party'])]['total_raised'] += float(group['sum_raised'])
    data[convert_party_code(group['filing_committee__party'])]['total_spent'] += float(group['sum_spent'])
    data[convert_party_code(group['filing_committee__party'])]['total_ie_for'] += float(group['sum_ie_for'] if group['sum_ie_for'] else 0)
    data[convert_party_code(group['filing_committee__party'])]['total_ie_against'] += float(group['sum_ie_against'] if group['sum_ie_against'] else 0)

for idx, group in enumerate(sorted(data.keys(), key=lambda k: data[k]['total_raised'], reverse=True)[:20]):
    print '%d) %s: %s raised' % (
        idx + 1,
        group,
        locale.currency(data[group]['total_raised'], grouping=True))

print '\n'
    
for idx, group in enumerate(sorted(data.keys(), key=lambda k: data[k]['total_spent'], reverse=True)[:20]):
    print '%d) %s: %s spent' % (
        idx + 1,
        group,
        locale.currency(data[group]['total_spent'], grouping=True))

print '\n'

for idx, group in enumerate(sorted(data.keys(), key=lambda k: data[k]['total_ie_for'], reverse=True)[:20]):
    print '%d) %s: %s in IEs supporting' % (
        idx + 1,
        group,
        locale.currency(data[group]['total_ie_for'], grouping=True))

print '\n'

for idx, group in enumerate(sorted(data.keys(), key=lambda k: data[k]['total_ie_against'], reverse=True)[:20]):
    print '%d) %s: %s in IEs opposing' % (
        idx + 1,
        group,
        locale.currency(data[group]['total_ie_against'], grouping=True))

1) (Blank): $82,996,131.51 raised
2) Democratic: $69,132,051.09 raised
3) Republican: $42,983,661.33 raised
4) Other: $2,043,087.51 raised
5) Nonpartisan: $1,063,639.68 raised
6) Libertarian: $59,808.90 raised
7) Independent: $23,473.15 raised
8) Green: $12,861.42 raised
9) Socialist: $0.00 raised
10) Constitutional: $0.00 raised


1) (Blank): $69,398,293.23 spent
2) Democratic: $57,003,184.95 spent
3) Republican: $36,811,471.35 spent
4) Other: $1,912,514.80 spent
5) Nonpartisan: $957,471.82 spent
6) Libertarian: $47,362.56 spent
7) Independent: $26,302.25 spent
8) Green: $3,949.93 spent
9) Socialist: $0.00 spent
10) Constitutional: $0.00 spent


1) Democratic: $3,312,506.46 in IEs supporting
2) Republican: $2,170,418.07 in IEs supporting
3) (Blank): $1,306,904.93 in IEs supporting
4) Other: $404,848.90 in IEs supporting
5) Nonpartisan: $303,305.61 in IEs supporting
6) Independent: $0.00 in IEs supporting
7) Socialist: $0.00 in IEs supporting
8) Constitutional: $0.00 in IEs supporting


In [9]:
all_contribs = ScrapedContribution.objects.all().aggregate(Sum('amount'), Count('donor'))
print '%s in %s contributions' % (
    locale.currency(all_contribs['amount__sum'], grouping=True),
    locale.format('%d', all_contribs['donor__count'], grouping=True))

$163,493,705.74 in 481,453 contributions


In [10]:
all_expenses = ScrapedExpense.objects.all().aggregate(Sum('amount'), Count('vendor'))
print '%s in %s expenses' % (
    locale.currency(all_expenses['amount__sum'], grouping=True),
    locale.format('%d', all_expenses['vendor__count'], grouping=True))

$153,453,765.02 in 79,174 expenses


In [11]:
cmtes = ScrapedTotals.objects.filter(filing_committee__committee_group='leg').values(
    'filing_committee__name').annotate(sum=Sum('raised'))
for group in sorted(cmtes, key=lambda k: k['sum'], reverse=True)[:20]:
    full_cmte = ScrapedCommittee.objects.filter(committee_group='leg', name=group['filing_committee__name'])
    maps = SOSPDCMap.objects.filter(pdc_filer_id__in=[fc.filer_id for fc in full_cmte])
    for m in maps:
        summaries = BallotRaceSummary.objects.filter(ballot_id=m.sos_ballot_id)
        districts = set([s.district for s in summaries])
        names = []
        for d in districts:
            names.append(d.district_name)
    print '%s (%s - %s; %s): %s raised' % (
        group['filing_committee__name'],
        full_cmte[0].party,
        full_cmte[0].office,
        ('; '.join(set(names))).replace('Legislative District ', 'LD-'),
        locale.currency(group['sum'], grouping=True))

LITZOW STEPHEN R (R - STATE SENATOR; LD-41): $795,346.51 raised
WILSON LYNDA D (R - STATE SENATOR; LD-17): $616,196.54 raised
MULLET MARK D (D - STATE SENATOR; LD-5): $552,136.30 raised
O'BAN STEVEN T (R - STATE SENATOR; LD-28): $551,614.57 raised
MAGENDANZ CHAD L (R - STATE SENATOR; LD-5): $517,576.69 raised
BAILEY BARBARA F (R - STATE SENATOR; LD-10): $485,449.29 raised
PROBST TIMOTHY P (O - STATE SENATOR; LD-17): $482,593.48 raised
WELLMAN LISA (D - STATE SENATOR; LD-41): $471,448.84 raised
SCHOESLER MARK G (R - STATE SENATOR; LD-9): $419,754.88 raised
PELOQUIN MARISA K (D - STATE SENATOR; LD-28): $402,057.20 raised
KILDUFF CHRISTINE J (D - STATE REPRESENTATIVE; LD-28): $360,330.07 raised
RIVERS ANNA M (R - STATE SENATOR; LD-18): $349,697.50 raised
HICKEL TERI H (R - STATE REPRESENTATIVE; LD-30): $329,228.30 raised
GRAVES PAUL S (R - STATE REPRESENTATIVE; LD-5): $318,828.00 raised
KING CURTIS P (R - STATE SENATOR; LD-14): $310,621.31 raised
SULLIVAN PATRICK J (D - STATE REPRESENTATI

In [12]:
cmtes = ScrapedTotals.objects.filter(filing_committee__committee_group='loc').values(
    'filing_committee__name').annotate(sum=Sum('raised'))
for group in sorted(cmtes, key=lambda k: k['sum'], reverse=True)[:20]:
    full_cmte = ScrapedCommittee.objects.filter(committee_group='loc', name=group['filing_committee__name'])
    maps = SOSPDCMap.objects.filter(pdc_filer_id__in=[fc.filer_id for fc in full_cmte])
    for m in maps:
        summaries = BallotRaceSummary.objects.filter(ballot_id=m.sos_ballot_id)
        districts = set([s.district for s in summaries])
        names = []
        for d in districts:
            names.append(d.district_name)
    print '%s (%s - %s; %s): %s raised' % (
        group['filing_committee__name'],
        full_cmte[0].party,
        full_cmte[0].office,
        ('; '.join(set(names))).replace('Legislative District ', 'LD-'),
        locale.currency(group['sum'], grouping=True))

DAMMEIER BRUCE F (R - COUNTY EXECUTIVE; COUNTY): $507,592.08 raised
TALBERT RICK B (D - COUNTY EXECUTIVE; COUNTY): $249,375.90 raised
O'QUINN SHELLY J (R - COUNTY COMMISSIONER; COUNTY WIDE; CO DISTRICT 2): $101,796.73 raised
ROACH DANIEL N (R - COUNTY EXECUTIVE; COUNTY WIDE; CO DISTRICT 2): $99,183.50 raised
ROACH PAM (R - COUNTY COUNCIL MEMBER; COUNTY COUNCIL DISTRICT NO. 2): $95,322.75 raised
HULSE KELSEY L (D - COUNTY COMMISSIONER; County of Thurston; Commissioner District  No. 2): $92,631.95 raised
DUNSHEE HANS M (D - COUNTY COUNCIL MEMBER; County Council District 5): $90,839.68 raised
EDWARDS GARY P (NO - COUNTY COMMISSIONER; County of Thurston; Commissioner District  No. 2): $89,467.00 raised
MUMM CANDACE M (D - COUNTY COMMISSIONER; CO DISTRICT 1; COUNTY WIDE): $85,616.50 raised
LOW SAMUEL E (R - COUNTY COUNCIL MEMBER; County Council District 5): $81,385.48 raised
COOPER JAMES (JIM) M JR (D - COUNTY COMMISSIONER; Commissioner District  No. 1; County of Thurston): $73,340.31 raise

In [13]:
donors = ScrapedContribution.objects.all().values('donor', 'city', 'state').annotate(
    sum=Sum('amount'), num=Count('donor'))
for group in sorted(donors, key=lambda k: k['sum'], reverse=True)[:20]:
    print '%s of %s, %s: %s in %s contribs' % (
        group['donor'],
        group['city'],
        group['state'],
        locale.currency(group['sum'], grouping=True),
        locale.format('%d', group['num'], grouping=True))

SEIU WASHINGTON STATE COUNCIL of SEATTLE, WA: $2,441,000.00 in 24 contribs
THE LEADERSHIP COUNCIL of OLYMPIA, WA: $2,383,000.00 in 42 contribs
HARRY TRUMAN FUND of SEATTLE, WA: $2,089,900.00 in 67 contribs
SEIU 775 of SEATTLE, WA: $1,822,000.00 in 19 contribs
HANAUER NICOLAS of SEATTLE, WA: $1,818,000.00 in 20 contribs
WASHINGTON STATE DEMOCRATIC CENTRAL COMMITTEE of SEATTLE, WA: $1,808,528.47 in 37 contribs
KENNEDY FUND of SEATTLE, WA: $1,415,850.00 in 57 contribs
WASHINGTON STATE REPUBLICAN PARTY of BELLEVUE, WA: $1,406,175.00 in 31 contribs
THE REAGAN FUND of OLYMPIA, WA: $1,367,500.00 in 23 contribs
BALLMER CONNIE of HUNTS POINT, WA: $1,002,000.00 in 5 contribs
REPRESENT US of FLORENCE, MA: $960,300.00 in 7 contribs
REPUBLICAN STATE LEADERSHIP CO of WASHINGTON, DC: $900,000.00 in 11 contribs
ENTERPRISE WASHINGTON'S JOBSPAC of BELLEVUE, WA: $776,000.00 in 13 contribs
HOLLAND CLYDE of VANCOUVER, WA: $745,800.00 in 61 contribs
SENATE REPUBLICAN CAMPAIGN COMMITTEE of OLYMPIA, WA: $739,

In [14]:
donors = ScrapedContribution.objects.all().values('employer').annotate(sum=Sum('amount'), num=Count('donor'))
for group in sorted(donors, key=lambda k: k['sum'], reverse=True)[:20]:
    print '%s: %s in %s contribs' % (
        group['employer'],
        locale.currency(group['sum'], grouping=True),
        locale.format('%d', group['num'], grouping=True))

: $98,850,742.20 in 260,573 contribs
None: $8,221,979.65 in 7,451 contribs
NONE: $4,008,998.75 in 13,615 contribs
N/A: $3,739,242.55 in 23,311 contribs
SELF: $3,399,554.25 in 7,531 contribs
NOT EMPLOYED: $3,281,367.09 in 14,714 contribs
SELF EMPLOYED: $2,597,999.86 in 5,407 contribs
RETIRED: $2,267,686.39 in 8,570 contribs
SELF-EMPLOYED: $1,271,728.25 in 7,496 contribs
2ND AVE PARTNERS: $1,000,000.00 in 5 contribs
FISHER INVESTMENTS: $982,325.00 in 162 contribs
SECOND AVENUE PARTNERS: $857,150.00 in 33 contribs
MICROSOFT: $674,866.59 in 2,013 contribs
JS CAPITAL MANAGEMENT LLC: $575,000.00 in 3 contribs
HOLLAND PARTNERS GROUP: $537,000.00 in 40 contribs
INFORMATION REQUESTED: $405,391.70 in 1,670 contribs
UNIVERSITY OF WASHINGTON: $363,019.47 in 1,848 contribs
NA: $307,627.71 in 768 contribs
STATE OF WASHINGTON: $253,639.65 in 1,219 contribs
THE BALLMER GROUP: $253,000.00 in 5 contribs


In [15]:
donors = ScrapedContribution.objects.all().values('occupation').annotate(sum=Sum('amount'), num=Count('donor'))
for group in sorted(donors, key=lambda k: k['sum'], reverse=True)[:20]:
    print '%s: %s in %s contribs' % (
        group['occupation'],
        locale.currency(group['sum'], grouping=True),
        locale.format('%d', group['num'], grouping=True))

: $95,698,115.77 in 248,788 contribs
RETIRED: $11,951,619.17 in 50,198 contribs
None: $7,263,486.00 in 4,304 contribs
ATTORNEY: $4,507,017.44 in 15,518 contribs
CEO: $3,257,572.45 in 2,132 contribs
NOT EMPLOYED: $2,117,129.47 in 12,720 contribs
VENTURE CAPITALIST: $1,943,200.00 in 63 contribs
INVESTOR: $1,901,549.32 in 556 contribs
HOMEMAKER: $1,353,140.68 in 3,563 contribs
CONSULTANT: $1,245,407.86 in 3,432 contribs
OWNER: $1,189,807.00 in 2,168 contribs
PRESIDENT: $1,177,305.68 in 1,895 contribs
PHYSICIAN: $1,071,707.21 in 3,416 contribs
EXECUTIVE: $1,048,826.09 in 1,375 contribs
CONDUIT TOTAL LISTED IN AGG. FIELD: $1,015,372.39 in 3,104 contribs
COMMUNITY VOLUNTEER: $681,641.82 in 119 contribs
MANAGER: $623,429.19 in 1,823 contribs
FOUNDER: $484,888.14 in 129 contribs
ENGINEER: $460,622.42 in 1,553 contribs
CHIEF EXECUTIVE OFFICER: $437,257.33 in 55 contribs
